In [1]:
import os
# Setting temp dir to /srv/data directory, as it otherwise fills up the home directory too much
# Just comment out on machines that are not "Goober"
os.environ["TMPDIR"] = "/srv/data/tmp"
os.makedirs("/srv/data/tmp", exist_ok=True)

%pip install haystack-ai
%pip install nltk
%pip install markdown-it-py
%pip install mdit_plain
%pip install openai
%pip install pandas
%pip install hf_xet
%pip install ollama-haystack==2.4.2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pipelines.preprocessing_pipelines.markdown_pipeline import pipeline as markdown_pipeline
from pipelines.preprocessing_pipelines.preprocessing_pipeline import pipeline as preprocessing_pipeline
from haystack.dataclasses import ByteStream
from utils.markdown_utils import for_each_markdown_file
from tqdm import tqdm
import pandas as pd
import uuid

tqdm.pandas()

## Test Data

### Preprocessing

In [3]:

data = []

def preprocess_markdown(filename: str, bytes):
    bytestream = ByteStream(data=bytes)
    file_id = str(uuid.uuid4())
    md_documents = markdown_pipeline.run({
        "markdown_converter" : {
            "byte_streams": [bytestream],
            "id": file_id,
            "meta": {
                "title": filename
            }
        }
    })["markdown_converter"]["documents"]
    print(md_documents)
    preprocessed_documents = preprocessing_pipeline.run({
        "splitter": { "documents": md_documents }
    })["filter"]["documents"]
    data.extend([{ "document": document } for document in preprocessed_documents])

for_each_markdown_file("./data", preprocess_markdown)

df = pd.DataFrame(data)

Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 262.92it/s]


[Document(id=7ae1e3f0-dd60-4371-ab7b-aeeebef45f64, content: 'Weshalb wird eine äußere Wendung in Erwägung gezogen?

Im Laufe der Schwangerschaft kann das ungebor...', meta: {'title': 'Äußere Wendung.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 175.43it/s]


[Document(id=cec11350-e783-4477-b76e-bd9afa5342c5, content: 'Ein Kaiserschnitt kann aus unterschiedlichen Gründen und mit variierender Dringlichkeit notwendig we...', meta: {'title': 'Kaiserschnitt.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 136.46it/s]


...

[Document(id=73715a31-955e-47df-a9fb-6c900290bddf, content: 'Die Geburt stellt einen natürlichen Ablauf dar. Durch moderne geburtshilfliche Verfahren, insbesonde...', meta: {'title': 'Medizinische Maßnahmen rund um die Geburt.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 166.52it/s]


[Document(id=5f228e4f-ab5c-4585-8ee9-bb4ce764dbbc, content: 'Damit Sie den geplanten Eingriff schmerzfrei erleben, stehen zwei grundlegende Methoden zur Verfügun...', meta: {'title': 'Narkose.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 407.13it/s]

[Document(id=f534ef37-3ee9-4d29-a4d5-7c7723f610a8, content: 'In bestimmten Situationen wird ein vorzeitiger Geburtsbeginn medizinisch empfohlen oder notwendig:

...', meta: {'title': 'Einleitung der Geburt.md'})]


In [4]:
len(df)

321

In [5]:
document_ids = df["document"].apply(lambda doc: doc.id)
is_unique = document_ids.is_unique

print("All document IDs are unique:", is_unique)


All document IDs are unique: True


In [6]:
df

,document
0,Document(id=cd20e2851eabc57193d0701ee1feb0bb91...
1,Document(id=064cc49648b2da93bbbdf3c5eff49d64d5...
2,Document(id=8cf370d93ab8a2d239a9733288e705d0f4...
3,Document(id=829095ebf88b7c5b70c985ac154e04a26b...
4,Document(id=ea0c518b7c51e474b09770e8db97096528...
...,...
316,Document(id=119e747e2ca00879df293e1d95a9980807...
317,Document(id=2b54490d2dc389c78578395ee772450ae8...
318,Document(id=26f240814fab011db71d880777b22052f0...
319,Document(id=66e835839865a6c9b9670f607dc5a34dc2...


In [7]:
df.to_pickle("data/all_documents.pkl")